In [116]:
import pandas as pd

df = pd.read_excel(r"D:\Documents\GitHub\Final_Project\output.xlsx")
reference = pd.read_csv(r"D:\Documents\GitHub\Final_Project\reference.csv")


In [58]:
pip install Levenshtein

   ---------------------------------------- 0.0/100.7 kB ? eta -:--:--
   ------------------------------------ --- 92.2/100.7 kB ? eta -:--:--
   ---------------------------------------- 100.7/100.7 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ----- ---------------------------------- 0.2/1.6 MB 4.1 MB/s eta 0:00:01
   ----------------- ---------------------- 0.7/1.6 MB 5.1 MB/s eta 0:00:01Note: you may need to restart the kernel to use updated packages.

   ------------------------- -------------- 1.0/1.6 MB 5.4 MB/s eta 0:00:01
   ------------------------------ --------- 1.2/1.6 MB 5.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.4/1.6 MB 5.4 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 5.2 MB/s eta 0:00:00


In [117]:
#Correct columns
df = df.rename(columns={"champignonmorille": "champignon"})

In [118]:
keywords = df.columns[8:]
keywords

Index(['boeuf', 'veau', 'porc', 'canard', 'lapin', 'poulet', 'crevettes',
       'moules', 'huitres', 'cabillaud',
       ...
       'orge', 'chocolat', 'fromage', 'avoine', 'huiledolive', 'huiledepalme',
       'huiledecolza', 'huiledesoja', 'huiledetournesol', 'seigle'],
      dtype='object', length=148)

In [119]:
#Prepare the df
#copie the df
#backup = df["product"]
df['product'] = df['product'].str.lower().str.replace(' ', '_')
import unicodedata
#remove accent
def remove_accents(text):
    return ''.join(c for c in unicodedata.normalize('NFKD', text) if not unicodedata.combining(c))
df['product'] = df['product'].apply(remove_accents)

In [ ]:
#preparation reference
import re

#Handle exception
df['product'] = df['product'].apply(lambda x: "chocolat" if re.search("choco", x, re.IGNORECASE) else x)
df['product'] = df['product'].apply(lambda x: "chocolat" if re.search("cacao", x, re.IGNORECASE) else x)
df['product'] = df['product'].apply(lambda x: "fromage" if re.search("brie", x, re.IGNORECASE) else x)
df['product'] = df['product'].apply(lambda x: "fromage" if re.search("emmental", x, re.IGNORECASE) else x)
df['product'] = df['product'].apply(lambda x: "fromage" if re.search("chevre", x, re.IGNORECASE) else x)
df['product'] = df['product'].apply(lambda x: "fromage" if re.search("chedar", x, re.IGNORECASE) else x)

In [121]:

from Levenshtein import ratio 

# Function to check partial match and update DataFrame
def mark_keywords(row):
    for keyword in keywords:
        for word in row['product'].split('_'):  # Split product name into words
            if re.search(keyword, word, re.IGNORECASE) or ratio(keyword, word) > 0.85:  # Allow slight variations
                row[keyword] = 'X'
    return row

# Apply function to each row
df_result = df.apply(mark_keywords, axis=1)

print(df)

                          rayons                             product  \
0               Epicerie sucrée                             chocolat   
1               Epicerie sucrée                       house_tassimo_   
2                       Boissons          biere_blonde_stella_artois   
3                       Boissons                        soda_selecto   
4             Fruits et Légumes   bananes_max_havelaar_carrefour_bio   
5             Fruits et Légumes                   champignons_blancs   
6  Crèmerie et Produits laitiers                             fromage   
7  Crèmerie et Produits laitiers                             fromage   

      poids(g)  prix_1  prix_2   prix  prix kilos/l  \
0   184.948980     NaN     NaN   1.45          7.84   
1   268.055556     5.0    79.0   5.79         21.60   
2  5009.803922    10.0    22.0  10.22          2.04   
3  1494.845361     NaN     NaN   1.45          0.97   
4   360.000000     NaN     NaN    NaN         16.58   
5   200.000000     Na

In [ ]:
#Put back the original titre
df_result["product"] = pd.read_excel(r"D:\Documents\GitHub\Final_Project\output.xlsx")['product']
#rename final file with day + hour saved

#Create the final file
df_result.to_excel("sample_result.xlsx", index=False)

In [90]:
df_result = pd.read_excel(r"D:\Documents\GitHub\Final_Project\sample_result.xlsx")

In [122]:
def get_footprint(row):
    for column in df_result.columns:
        # Check if the value in df is not empty (not None or NaN)
        if pd.notna(row[column]):
            # Find the corresponding footprint in the 'reference' DataFrame
            footprint_value = reference.loc[reference['slug'] == column, 'footprint'].values
            if footprint_value.size > 0:
                return footprint_value[0]  # Return the footprint value for the matched product
    return None  # If no non-empty values are found


# Apply the function to each row in df
df_result['footprint'] = df_result.apply(get_footprint, axis=1)

#Framing
def get_farming(row):
    for column in df_result.columns:
        # Check if the value in df is not empty (not None or NaN)
        if pd.notna(row[column]):
            # Find the corresponding footprint in the 'reference' DataFrame
            farming_value = reference.loc[reference['slug'] == column, 'farming'].values
            if farming_value.size > 0:
                return farming_value[0]  # Return the footprint value for the matched product
    return None  # If no non-empty values are found


# Apply the function to each row in df
df_result['farming'] = df_result.apply(get_farming, axis=1)

#Processing
def get_processing(row):
    for column in df_result.columns:
        # Check if the value in df is not empty (not None or NaN)
        if pd.notna(row[column]):
            # Find the corresponding footprint in the 'reference' DataFrame
            processing_value = reference.loc[reference['slug'] == column, 'processing'].values
            if processing_value.size > 0:
                return processing_value[0]  # Return the footprint value for the matched product
    return None  # If no non-empty values are found


# Apply the function to each row in df
df_result['processing'] = df_result.apply(get_processing, axis=1)

#Packaging
def get_packaging(row):
    for column in df_result.columns:
        # Check if the value in df is not empty (not None or NaN)
        if pd.notna(row[column]):
            # Find the corresponding footprint in the 'reference' DataFrame
            packaging_value = reference.loc[reference['slug'] == column, 'packaging'].values
            if packaging_value.size > 0:
                return packaging_value[0]  # Return the footprint value for the matched product
    return None  # If no non-empty values are found


# Apply the function to each row in df
df_result['packaging'] = df_result.apply(get_packaging, axis=1)

#Retail
def get_retail(row):
    for column in df_result.columns:
        # Check if the value in df is not empty (not None or NaN)
        if pd.notna(row[column]):
            # Find the corresponding footprint in the 'reference' DataFrame
            retail_value = reference.loc[reference['slug'] == column, 'retail'].values
            if retail_value.size > 0:
                return retail_value[0]  # Return the footprint value for the matched product
    return None  # If no non-empty values are found


# Apply the function to each row in df
df_result['retail'] = df_result.apply(get_retail, axis=1)

#Transport
def get_transport(row):
    for column in df_result.columns:
        # Check if the value in df is not empty (not None or NaN)
        if pd.notna(row[column]):
            # Find the corresponding footprint in the 'reference' DataFrame
            transport_value = reference.loc[reference['slug'] == column, 'transport'].values
            if transport_value.size > 0:
                return transport_value[0]  # Return the footprint value for the matched product
    return None  # If no non-empty values are found


# Apply the function to each row in df
df_result['transport'] = df_result.apply(get_transport, axis=1)

#Consumption
def get_consumption(row):
    for column in df_result.columns:
        # Check if the value in df is not empty (not None or NaN)
        if pd.notna(row[column]):
            # Find the corresponding footprint in the 'reference' DataFrame
            consumption_value = reference.loc[reference['slug'] == column, 'consumption'].values
            if consumption_value.size > 0:
                return consumption_value[0]  # Return the footprint value for the matched product
    return None  # If no non-empty values are found


# Apply the function to each row in df
df_result['consumption'] = df_result.apply(get_consumption, axis=1)

C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_12100\1175754737.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result['footprint'] = df_result.apply(get_footprint, axis=1)
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_12100\1175754737.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result['farming'] = df_result.apply(get_farming, axis=1)
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_12100\1175754737.py:43: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fram

In [123]:
df_result['total_kg_co2'] = df_result['footprint'] * df["poids(g)"] /1000
df_result['total_kg_co2_farming'] = df_result['farming'] * df["poids(g)"] /1000
df_result['total_kg_co2_processing'] = df_result['processing'] * df["poids(g)"] /1000
df_result['total_kg_co2_packaging'] = df_result['packaging'] * df["poids(g)"] /1000
df_result['total_kg_co2_transport'] = df_result['transport'] * df["poids(g)"] /1000
df_result['total_kg_co2_retail'] = df_result['retail'] * df["poids(g)"] /1000
df_result['total_kg_co2_consumption'] = df_result['consumption'] * df["poids(g)"] /1000

C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_12100\1352882074.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result['total_kg_co2'] = df_result['footprint'] * df["poids(g)"] /1000
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_12100\1352882074.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_result['total_kg_co2_farming'] = df_result['farming'] * df["poids(g)"] /1000
C:\Users\UTILISATEUR\AppData\Local\Temp\ipykernel_12100\1352882074.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually 

In [124]:
df_result

,rayons,product,poids(g),prix_1,prix_2,prix,prix kilos/l,link,boeuf,veau,...,retail,transport,consumption,total_kg_co2,total_kg_co2_farming,total_kg_co2_processing,total_kg_co2_packaging,total_kg_co2_transport,total_kg_co2_retail,total_kg_co2_consumption
0,Epicerie sucrée,chocolat,184.948980,NaN,NaN,1.45,7.84,https://www.carrefour.fr/p/biscuits-choco-brow...,NaN,NaN,...,0.0370,0.1108,NaN,1.459210,1.236754,0.061588,0.133533,0.020492,0.006843,NaN
1,Epicerie sucrée,house_tassimo_,268.055556,5.0,79.0,5.79,21.60,https://www.carrefour.fr/p/cafe-dosettes-latte...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Boissons,biere_blonde_stella_artois,5009.803922,10.0,22.0,10.22,2.04,https://www.carrefour.fr/p/biere-blonde-stella...,NaN,NaN,...,0.0363,0.3300,0.00683,6.748356,0.400283,1.367676,3.111088,1.653235,0.181856,0.034217
3,Boissons,soda_selecto,1494.845361,NaN,NaN,1.45,0.97,https://www.carrefour.fr/p/soda-selecto-337743...,NaN,NaN,...,0.0363,0.2090,0.00683,0.737452,0.134835,0.019433,0.206289,0.312423,0.054263,0.010210
4,Fruits et Légumes,bananes_max_havelaar_carrefour_bio,360.000000,NaN,NaN,NaN,16.58,https://www.carrefour.fr/p/bananes-max-havelaa...,NaN,NaN,...,0.0284,0.4470,0.13800,0.327068,0.106200,0.000044,NaN,0.160920,0.010224,0.049680
5,Fruits et Légumes,champignons_blancs,200.000000,NaN,NaN,0.99,4.95,https://www.carrefour.fr/p/champignons-blancs-...,NaN,NaN,...,0.0560,0.2980,0.08060,1.235002,1.148000,0.000082,NaN,0.059600,0.011200,0.016120
6,Crèmerie et Produits laitiers,fromage,250.000000,2.0,29.0,2.29,9.16,https://www.carrefour.fr/p/emmental-francais-c...,NaN,NaN,...,0.0368,0.2556,0.00680,1.465650,1.216950,0.067825,0.106075,0.063900,0.009200,0.001700
7,Crèmerie et Produits laitiers,fromage,200.000000,1.0,49.0,1.49,7.45,https://www.carrefour.fr/p/pointe-de-brie-crem...,NaN,NaN,...,0.0368,0.2556,0.00680,1.172520,0.973560,0.054260,0.084860,0.051120,0.007360,0.001360


In [109]:
df_result["product"] = pd.read_excel(r"D:\Documents\GitHub\Final_Project\output.xlsx")['product']

In [110]:
df_result

,rayons,product,poids(g),prix_1,prix_2,prix,prix kilos/l,link,boeuf,veau,...,huiledolive,huiledepalme,huiledecolza,huiledesoja,huiledetournesol,seigle,footprint,farming,total_kg_co2,total_kg_co2_farming
0,Epicerie sucrée,Biscuits Choco Brownies BN,184.948980,NaN,NaN,1.45,7.84,https://www.carrefour.fr/p/biscuits-choco-brow...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.889800,6.6870,1.459210,1.236754
1,Epicerie sucrée,HOUSE TASSIMO,268.055556,5.0,79.0,5.79,21.60,https://www.carrefour.fr/p/cafe-dosettes-latte...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Boissons,Bière Blonde STELLA ARTOIS,5009.803922,10.0,22.0,10.22,2.04,https://www.carrefour.fr/p/biere-blonde-stella...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.347030,0.0799,6.748356,0.400283
3,Boissons,Soda SELECTO,1494.845361,NaN,NaN,1.45,0.97,https://www.carrefour.fr/p/soda-selecto-337743...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.493330,0.0902,0.737452,0.134835
4,Fruits et Légumes,Bananes Max Havelaar CARREFOUR BIO,360.000000,NaN,NaN,NaN,16.58,https://www.carrefour.fr/p/bananes-max-havelaa...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.908523,0.2950,0.327068,0.106200
5,Fruits et Légumes,Champignons blancs,200.000000,NaN,NaN,0.99,4.95,https://www.carrefour.fr/p/champignons-blancs-...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Crèmerie et Produits laitiers,Emmental français CARREFOUR CLASSIC',250.000000,2.0,29.0,2.29,9.16,https://www.carrefour.fr/p/emmental-francais-c...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.862600,4.8678,1.465650,1.216950
7,Crèmerie et Produits laitiers,Pointe de Brie crémeux CARREFOUR CLASSIC',200.000000,1.0,49.0,1.49,7.45,https://www.carrefour.fr/p/pointe-de-brie-crem...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.862600,4.8678,1.172520,0.973560


In [125]:
df_result.to_excel("sample_result.xlsx", index=False)